# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

You can use ** Keras ** to implement your model. Read more at [keras.io](https://keras.io/).

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0)). You are not expected to model your architecture precisely using this model nor get the same performance levels, but this is more to show an exampe of an approach used to solve this particular problem. We encourage you to try out different architectures for yourself and see what works best for you. Here is a useful [forum post](https://discussions.udacity.com/t/goodfellow-et-al-2013-architecture/202363) discussing the architecture as described in the paper and here is [another one](https://discussions.udacity.com/t/what-loss-function-to-use-for-multi-digit-svhn-training/176897) discussing the loss function.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [24]:
from __future__ import print_function

import PIL.Image
from io import BytesIO
from IPython.display import Image, display

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

from six.moves import cPickle as pickle
from six.moves import range
#from six.moves import urllib
#from six.moves import xrange

import gzip
import os
import tempfile

# Could not download dataset, site is down.
# from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
# data_dir = ''
# mnist = read_data_sets(data_dir, one_hot=True)



### Your code implementation goes here.
### Feel free to use as many code cells as needed.

## Load notMNIST pickle

In [9]:

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Make data gray scale

In [10]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [53]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)
        
def output_size_pool(input_size, conv_filter_size, pool_filter_size,
                         padding, conv_stride, pool_stride):
        if padding == 'same':
            padding = -1.00
        elif padding == 'valid':
            padding = 0.00
        else:
            return None
        # After convolution 1
        output_1 = (
            ((input_size - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        # After pool 1
        output_2 = (
            ((output_1 - pool_filter_size - 2 * padding) / pool_stride) + 1.00)
        # After convolution 2
        output_3 = (
            ((output_2 - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        # After pool 2
        output_4 = (
            ((output_3 - pool_filter_size - 2 * padding) / pool_stride) + 1.00)
        # After convolution 2
        output_5 = (
            ((output_4 - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        # After pool 2
        # output_6 = (
        #     ((output_5 - pool_filter_size - 2 * padding) / pool_stride) + 1.00)
        return int(output_5)

## Create Computational graph

In [64]:
batch_size = 16
patch_size = 5

depth_1 = 16
depth_2 = depth_1 * 2
depth_3 = depth_2 * 3

num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  keep_prob = tf.placeholder(tf.float32)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = weight_variable([patch_size, patch_size, num_channels, depth_1])
  variable_summaries(layer1_weights)
  layer1_biases = tf.Variable(tf.constant(0.1, shape=[depth_1])) # tf.zeros([depth]))

  layer2_weights = weight_variable([patch_size, patch_size, depth_1, depth_2])
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))

  layer3_weights = weight_variable([patch_size, patch_size, depth_2, depth_3])
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth_3]))

  final_image_size = output_size_pool(input_size=image_size,
                                        conv_filter_size=5, pool_filter_size=2,
                                        padding='same', conv_stride=1,
                                        pool_stride=2)
    

  layer4_weights = weight_variable([final_image_size * final_image_size * depth_3, num_hidden])
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))


  logits1_weights = weight_variable([num_hidden, num_labels])
  logits1_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  logits2_weights = weight_variable([num_hidden, num_labels])
  logits2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  logits3_weights = weight_variable([num_hidden, num_labels])
  logits3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  logits4_weights = weight_variable([num_hidden, num_labels])
  logits4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  logits5_weights = weight_variable([num_hidden, num_labels])
  logits5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    # conv 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden_relu = tf.nn.relu(conv + layer1_biases)
    hidden_polling = tf.nn.max_pool(hidden_relu, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
    # conv 2
    conv = tf.nn.conv2d(hidden_polling, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_relu = tf.nn.relu(conv + layer2_biases)
    hidden_polling = tf.nn.max_pool(hidden_relu, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
    # conv 3
    conv = tf.nn.conv2d(hidden_polling, layer3_weights, [1, 1, 1, 1], padding='SAME')
    hidden_relu = tf.nn.relu(conv + layer3_biases)
    ###
    
    # fc
    dropout = tf.nn.dropout(hidden_relu, keep_prob)
    
    shape = dropout.get_shape().as_list()
    reshape = tf.reshape(dropout, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_fc = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    
    return hidden_fc
  
  # Training computation.
  #[logits_1, logits_2, logits_3, logits_4, logits_5] = model(tf_train_dataset)
  fc = model(tf_train_dataset)
  logits_1 = tf.matmul(fc, logits1_weights) + logits1_biases
  logits_2 = tf.matmul(fc, logits2_weights) + logits2_biases
  logits_3 = tf.matmul(fc, logits3_weights) + logits3_biases
  logits_4 = tf.matmul(fc, logits4_weights) + logits4_biases
  logits_5 = tf.matmul(fc, logits5_weights) + logits5_biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_1))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


ValueError: Dimensions must be equal, but are 4704 and 2400 for 'MatMul' (op: 'MatMul') with input shapes: [16,4704], [2400,64].

In [51]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels,
        keep_prob: 1.0
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict={ keep_prob: 1.0 } ), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={ keep_prob: 1.0 } ), test_labels))


# 1. Test accuracy: 89.8%
# 2. Test accuracy: 90.4%
# 3. Test accuracy: 89.6% / 90%

Initialized
Minibatch loss at step 0: 5.264670
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.737583
Minibatch accuracy: 37.5%
Validation accuracy: 53.7%
Minibatch loss at step 100: 1.299867
Minibatch accuracy: 56.2%
Validation accuracy: 66.7%
Minibatch loss at step 150: 0.385854
Minibatch accuracy: 81.2%
Validation accuracy: 73.0%
Minibatch loss at step 200: 1.209493
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 250: 1.331345
Minibatch accuracy: 62.5%
Validation accuracy: 75.6%
Minibatch loss at step 300: 0.426129
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.417387
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.270623
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.826538
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.614401
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
M

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.